In [1]:

print("📦 Installing packages and fixing compatibility issues...")

# Fix protobuf compatibility issue
!pip uninstall -y protobuf
!pip install -q protobuf==3.20.3

# Install required packages
!pip install -q transformers datasets accelerate sentencepiece evaluate rouge_score

print("\n✅ Installation complete!")
print("⚠️  Ignore any dependency warnings - they won't affect training")
print("🔄 If you see CUDA warnings above, that's normal - training will still work")


📦 Installing packages and fixing compatibility issues...
Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.4 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incomp

In [3]:
# =====================================================================
# CELL 2: Imports & Configuration (Run Second, ~10 seconds)
# =====================================================================

import os
import torch
import pandas as pd
from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
import numpy as np
from tqdm.auto import tqdm
import time
import json
from collections import Counter

print("✅ All imports successful!")

# Configuration
CONFIG = {
    # Model settings
    "model_name": "google/flan-t5-base",
    "output_dir": "./academic-summarizer-scientific",
    
    # Mixed Dataset Strategy (70-20-10 split)
    "scientific_samples": 20000,  # 70% - arXiv scientific papers
    "booksum_samples": 6000,      # 20% - Long-form book summaries
    "wikihow_samples": 2500,      # 10% - Instructional articles
    
    # Training hyperparameters
    "learning_rate": 3e-4,
    "num_epochs": 3,
    "batch_size": 8,
    "gradient_accumulation": 4,
    
    # Text settings
    "max_input_length": 1024,
    "max_target_length": 512,
    
    # Optimization
    "fp16": torch.cuda.is_available(),  # auto enable fp16 only if GPU
    "warmup_steps": 500,
}
CONFIG["max_input_length"] = 640
CONFIG["max_target_length"] = 160

print("\n🚀 Configuration:")
print("=" * 60)
print(f"Model: {CONFIG['model_name']}")
print(f"Dataset Mix: 70% Scientific + 20% BookSum + 10% WikiHow")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Count: {torch.cuda.device_count()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
print("=" * 60)


2025-11-08 16:10:40.005199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762618240.379367      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762618240.476270      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ All imports successful!

🚀 Configuration:
Model: google/flan-t5-base
Dataset Mix: 70% Scientific + 20% BookSum + 10% WikiHow
GPU Available: True
GPU Count: 2
GPU Name: Tesla T4


In [4]:
# =====================================================================
# CELL 3: Load Datasets (Run Third, ~5-10 minutes)
# =====================================================================

print("\n📥 Loading datasets (this takes 5-10 minutes)...\n")

# 1) Scientific Papers = ArXiv (Parquet-backed mirror)
print("1️⃣ Loading Scientific Papers (ArXiv)...")
scientific_dataset = load_dataset(
    "ccdv/arxiv-summarization",
    "document",
    split=f"train[:{CONFIG['scientific_samples']}]"
)
print(f"   ✅ Loaded {len(scientific_dataset):,} arXiv articles")

# 2) BookSum (auto-detect config)
from datasets import get_dataset_config_names
try:
    bs_configs = get_dataset_config_names("kmfoda/booksum")
except Exception:
    bs_configs = ["default"]
bs_config = bs_configs[0] if bs_configs else None
print(f"\n2️⃣ Loading BookSum (config: {bs_config})...")
booksum_dataset = load_dataset(
    "kmfoda/booksum",
    bs_config if bs_config else None,
    split=f"train[:{CONFIG['booksum_samples']}]"
)
print(f"   ✅ Loaded {len(booksum_dataset):,} BookSum rows")
print("   🧱 BookSum columns:", booksum_dataset.column_names)

# 3) WikiHow (cleaned HF copy)
print("\n3️⃣ Loading WikiHow (cleaned)...")
wikihow_dataset = load_dataset(
    "gursi26/wikihow-cleaned",
    split=f"train[:{CONFIG['wikihow_samples']}]"
)
print(f"   ✅ Loaded {len(wikihow_dataset):,} WikiHow rows")
print("   🧱 WikiHow columns:", wikihow_dataset.column_names)

print("\n✅ All datasets loaded successfully!")



📥 Loading datasets (this takes 5-10 minutes)...

1️⃣ Loading Scientific Papers (ArXiv)...


README.md: 0.00B [00:00, ?B/s]

document/train-00000-of-00015.parquet:   0%|          | 0.00/227M [00:00<?, ?B/s]

document/train-00001-of-00015.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

document/train-00002-of-00015.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

...

   ✅ Loaded 6,000 BookSum rows
   🧱 BookSum columns: ['bid', 'is_aggregate', 'source', 'chapter_path', 'summary_path', 'book_id', 'summary_id', 'content', 'summary', 'chapter', 'chapter_length', 'summary_name', 'summary_url', 'summary_text', 'summary_analysis', 'summary_length', 'analysis_length']

3️⃣ Loading WikiHow (cleaned)...


README.md:   0%|          | 0.00/792 [00:00<?, ?B/s]

wikihow-cleaned.csv:   0%|          | 0.00/619M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/214293 [00:00<?, ? examples/s]

   ✅ Loaded 2,500 WikiHow rows
   🧱 WikiHow columns: ['summary', 'title', 'text']

✅ All datasets loaded successfully!


In [5]:
# =====================================================================
# CELL 4: Process & Combine Datasets (Run Fourth, ~2-3 minutes)
# =====================================================================

print("\n🔄 Processing and combining datasets...")

# ---------------------------
# Scientific (arXiv) → uses: article, abstract
# ---------------------------
def process_scientific(example):
    return {
        "document": example["article"],
        "summary": example["abstract"],
        "source": "scientific",
    }

scientific_processed = scientific_dataset.map(
    process_scientific,
    remove_columns=scientific_dataset.column_names,
    desc="Processing Scientific Papers",
)

# ---------------------------
# BookSum → columns seen: content, chapter, summary_text, summary
# prefer 'chapter' if present, else 'content'; prefer 'summary_text' else 'summary'
# ---------------------------
BS_DOC_COL = "chapter" if "chapter" in booksum_dataset.column_names else "content"
BS_SUM_COL = "summary_text" if "summary_text" in booksum_dataset.column_names else "summary"

def process_booksum(example):
    return {
        "document": example[BS_DOC_COL],
        "summary": example[BS_SUM_COL],
        "source": "booksum",
    }

booksum_processed = booksum_dataset.map(
    process_booksum,
    remove_columns=booksum_dataset.column_names,
    desc="Processing BookSum",
)

# ---------------------------
# WikiHow (cleaned) → columns: text, summary, title
# ---------------------------
def process_wikihow(example):
    return {
        "document": example["text"],
        "summary": example["summary"],
        "source": "wikihow",
    }

wikihow_processed = wikihow_dataset.map(
    process_wikihow,
    remove_columns=wikihow_dataset.column_names,
    desc="Processing WikiHow",
)

# ---------------------------
# Combine → shuffle → split 90/10
# ---------------------------
combined_dataset = concatenate_datasets(
    [scientific_processed, booksum_processed, wikihow_processed]
).shuffle(seed=42)

dataset_split = combined_dataset.train_test_split(test_size=0.1, seed=42)

# use 'test' split as validation
dataset_dict = DatasetDict({
    "train": dataset_split["train"],
    "validation": dataset_split["test"],
})
dataset_dict["train"] = dataset_dict["train"].shuffle(seed=42).select(range(8000))
dataset_dict["validation"] = dataset_dict["validation"].shuffle(seed=42).select(range(800))
# ---------------------------
# Show distribution
# ---------------------------
sources = dataset_dict["train"]["source"]
from collections import Counter  # in case cell order changes
distribution = Counter(sources)

print("\n✅ Datasets combined!")
print(f"Train: {len(dataset_dict['train']):,}")
print(f"Validation: {len(dataset_dict['validation']):,}")
print("\n📈 Training Distribution:")
for source, count in distribution.items():
    print(f"  {source}: {count:,} ({count/len(sources)*100:.1f}%)")



🔄 Processing and combining datasets...


Processing Scientific Papers:   0%|          | 0/20000 [00:00<?, ? examples/s]

Processing BookSum:   0%|          | 0/6000 [00:00<?, ? examples/s]

Processing WikiHow:   0%|          | 0/2500 [00:00<?, ? examples/s]


✅ Datasets combined!
Train: 8,000
Validation: 800

📈 Training Distribution:
  scientific: 5,659 (70.7%)
  booksum: 1,653 (20.7%)
  wikihow: 688 (8.6%)


In [6]:
# =====================================================================
# CELL 5: Load Model & Tokenizer (Run Fifth, ~1 minute)
# =====================================================================

print("\n🤖 Loading T5 model and tokenizer...")

from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = CONFIG["model_name"]

# Tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Model
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Device info
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print("✅ Model loaded!")
print(f"   Name: {model_name}")
try:
    print(f"   Parameters: {model.num_parameters() / 1e6:.1f}M")
except Exception:
    pass

print(f"   Device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")



🤖 Loading T5 model and tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model loaded!
   Name: google/flan-t5-base
   Parameters: 247.6M
   Device: cuda
   GPU: Tesla T4


In [7]:
# =====================================================================
# CELL 6: Tokenize Datasets (Run Sixth, ~5-8 minutes)
# =====================================================================

print("\n🔄 Tokenizing datasets (this takes 5-8 minutes)...")

def preprocess_function(examples):
    
    # prefix instruction based on source
    prefixes = []
    for s in examples["source"]:
        if s == "scientific":
            prefixes.append("summarize scientific paper: ")
        elif s == "booksum":
            prefixes.append("summarize book chapter: ")
        else:
            prefixes.append("summarize instructions: ")
    
    inputs = [p + d for p, d in zip(prefixes, examples["document"])]
    targets = examples["summary"]

    # tokenize input documents
    model_inputs = tokenizer(
        inputs,
        max_length=CONFIG["max_input_length"],
        truncation=True,
        padding="max_length",
    )

    # tokenize summaries / labels
    labels = tokenizer(
        targets,
        max_length=CONFIG["max_target_length"],
        truncation=True,
        padding="max_length",
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset_dict.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
    desc="Tokenizing"
)

print("✅ Tokenization complete!")



🔄 Tokenizing datasets (this takes 5-8 minutes)...


Tokenizing:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/800 [00:00<?, ? examples/s]

✅ Tokenization complete!


In [9]:
# =====================================================================
# CELL 7C: Speed Mode (no re-tokenize) — finish within a short session
# =====================================================================
import gc, torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

print("⚡ Switching to speed mode...")

gc.collect(); torch.cuda.empty_cache()

# keep gradient checkpointing + no cache from 7B
try:
    model.gradient_checkpointing_enable()
except Exception:
    pass
model.config.use_cache = False

speed_args = Seq2SeqTrainingArguments(
    output_dir=CONFIG["output_dir"],
    # ⚠️ do fewer epochs
    num_train_epochs=1,
    optim="adafactor",
    # small per-device batch to avoid OOM, but large grad-accum to reduce steps/epoch
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=32,   # halves steps vs 16
    learning_rate=CONFIG["learning_rate"],
    warmup_steps=200,
    weight_decay=0.01,
    fp16=CONFIG["fp16"],
    # 🚫 turn off eval during training to save time
    eval_strategy="no",
    predict_with_generate=False,      # generation is slow; skip during train
    # avoid checkpoint overhead
    save_steps=10_000_000,
    save_total_limit=1,
    logging_steps=200,
    logging_dir=f"{CONFIG['output_dir']}/logs",
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    remove_unused_columns=True,
    report_to="none",
    group_by_length=True,
)

speed_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model, label_pad_token_id=-100, padding=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=speed_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],  # we won't evaluate during train
    processing_class=tokenizer,   # replaces tokenizer=tokenizer (silences warning)
    data_collator=speed_collator,
    compute_metrics=None,         # metrics disabled during train for speed
)

print("✅ Speed mode ready. Now run Cell 8 again.")


⚡ Switching to speed mode...
✅ Speed mode ready. Now run Cell 8 again.


In [10]:
# =====================================================================
# CELL 8: START TRAINING (Run Eighth — long)
# =====================================================================

import math, time

print("\n" + "=" * 60)
print("🚀 STARTING TRAINING")
print("=" * 60)

# Rough time estimate (purely informational)
total_samples = len(tokenized_dataset["train"])
effective_batch = CONFIG["batch_size"] * CONFIG["gradient_accumulation"]
steps_per_epoch = max(1, math.floor(total_samples / effective_batch))
total_steps = steps_per_epoch * CONFIG["num_epochs"]

print(f"Total train samples: {total_samples:,}")
print(f"Per-device batch size: {CONFIG['batch_size']}")
print(f"Gradient accumulation: {CONFIG['gradient_accumulation']}")
print(f"Effective batch size:  {effective_batch}")
print(f"Epochs: {CONFIG['num_epochs']}")
print(f"Estimated steps/epoch: {steps_per_epoch:,}")
print(f"Total steps (est.):    {total_steps:,}")
print("=" * 60 + "\n")

start_time = time.time()

# If you want to resume from the last checkpoint (if any), set this True
RESUME_LAST = False
if RESUME_LAST:
    last_ckpt = trainer.state.best_model_checkpoint or None
    trainer.train(resume_from_checkpoint=last_ckpt)
else:
    trainer.train()

end_time = time.time()
elapsed = end_time - start_time
h = int(elapsed // 3600)
m = int((elapsed % 3600) // 60)

print("\n" + "=" * 60)
print("✅ TRAINING COMPLETE!")
print("=" * 60)
print(f"⏱️  Wall-clock time: {h}h {m}m")
print("=" * 60)



🚀 STARTING TRAINING
Total train samples: 8,000
Per-device batch size: 8
Gradient accumulation: 4
Effective batch size:  32
Epochs: 3
Estimated steps/epoch: 250
Total steps (est.):    750



/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss



✅ TRAINING COMPLETE!
⏱️  Wall-clock time: 3h 36m


In [13]:
# MINI FIX: re-define compute_metrics if kernel lost it

import numpy as np
import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    pred_lens = [len(p.split()) for p in decoded_preds]
    result["gen_len"] = float(np.mean(pred_lens))

    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
        "avg_length": result["gen_len"],
    }
print("✅ compute_metrics restored.")


✅ compute_metrics restored.


In [14]:
# =====================================================================
# CELL 9: Evaluate Model (Run Ninth, ~2-3 minutes)
# =====================================================================

print("\n📊 Final Evaluation on Validation Set...")

import gc, torch
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# If you used Speed Mode (Cell 7C), trainer may have compute_metrics=None and
# predict_with_generate=False. For evaluation we rebuild a light eval-only trainer.
eval_args = Seq2SeqTrainingArguments(
    output_dir=CONFIG["output_dir"],
    per_device_eval_batch_size=2,
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    predict_with_generate=True,
    generation_max_length=min(192, CONFIG["max_target_length"]),
    generation_num_beams=2,                 # keep it light on GPU
    fp16=CONFIG["fp16"],
    report_to="none",
)

eval_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model, label_pad_token_id=-100, padding=True
)

eval_trainer = Seq2SeqTrainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_dataset["validation"],
    processing_class=tokenizer,             # (instead of tokenizer=...) silences warning
    data_collator=eval_collator,
    compute_metrics=compute_metrics,        # defined in Cell 7
)

# Run evaluation
eval_results = eval_trainer.evaluate()

print("\n✅ Evaluation Results:")
print("-" * 60)
for k in sorted(eval_results.keys()):
    if isinstance(eval_results[k], (int, float)):
        try:
            print(f"  {k}: {eval_results[k]:.4f}")
        except Exception:
            print(f"  {k}: {eval_results[k]}")
    else:
        print(f"  {k}: {eval_results[k]}")
print("-" * 60)



📊 Final Evaluation on Validation Set...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



✅ Evaluation Results:
------------------------------------------------------------
  eval_avg_length: 101.3700
  eval_loss: 2.5207
  eval_rouge1: 0.2933
  eval_rouge2: 0.0754
  eval_rougeL: 0.1989
  eval_runtime: 970.5007
  eval_samples_per_second: 0.8240
  eval_steps_per_second: 0.2060
------------------------------------------------------------


In [15]:
# =====================================================================
# CELL 10: Save & Quick Test (Run Tenth, ~2-3 minutes)
# =====================================================================

import json, time, torch

print("\n💾 Saving final model & tokenizer...")

output_path = "./my_academic_summarizer_scientific"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

# Collect training/eval info if available
try:
    _final_metrics = {k: float(v) for k, v in eval_results.items()}
except Exception:
    _final_metrics = {}

training_info = {
    "model": CONFIG["model_name"],
    "datasets": {
        "scientific_papers": int(CONFIG["scientific_samples"]),
        "booksum": int(CONFIG["booksum_samples"]),
        "wikihow": int(CONFIG["wikihow_samples"]),
        "total": int(CONFIG["scientific_samples"] + CONFIG["booksum_samples"] + CONFIG["wikihow_samples"]),
    },
    "final_metrics": _final_metrics,
    "training_epochs": int(getattr(trainer.args, "num_train_epochs", CONFIG["num_epochs"])),
    "per_device_train_batch_size": int(getattr(trainer.args, "per_device_train_batch_size", CONFIG["batch_size"])),
    "gradient_accumulation_steps": int(getattr(trainer.args, "gradient_accumulation_steps", CONFIG["gradient_accumulation"])),
    "max_input_length": int(CONFIG["max_input_length"]),
    "max_target_length": int(CONFIG["max_target_length"]),
    "timestamp": int(time.time()),
}

with open(f"{output_path}/training_info.json", "w") as f:
    json.dump(training_info, f, indent=2)

print(f"✅ Saved to: {output_path}")

# ---------------------------
# Quick smoke test on 2 samples
# ---------------------------
print("\n🧪 Quick generation test...\n")

device = model.device if hasattr(model, "device") else ("cuda" if torch.cuda.is_available() else "cpu")

test_examples = [
    {
        "type": "Course Policy",
        "text": (
            "Cloud Computing (CC-702IT0C026) is designed for senior undergraduates. "
            "It covers service models (IaaS, PaaS, SaaS), deployment models, virtualization, "
            "and cloud security. Networks is a prerequisite."
        ),
    },
    {
        "type": "Research Paper",
        "text": (
            "We propose federated learning for healthcare analytics, avoiding centralization "
            "of patient data. Experiments show improved accuracy with differential privacy."
        ),
    },
]

gen_max = min(256, CONFIG["max_target_length"])  # keep light for T4

for ex in test_examples:
    print("=" * 60)
    print(f"📄 {ex['type']}")
    print("=" * 60)

    prompt = "summarize scientific paper: " + ex["text"]

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=CONFIG["max_input_length"],
        truncation=True,
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=gen_max,
            num_beams=4,
            length_penalty=1.0,
            early_stopping=True,
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"📝 Summary: {summary}\n")

print("✅ Quick test complete. Ready for export.")



💾 Saving final model & tokenizer...
✅ Saved to: ./my_academic_summarizer_scientific

🧪 Quick generation test...

📄 Course Policy
📝 Summary: Cloud Computing (CC-702IT0C026) is designed for senior undergraduates and covers service models (IaaS, PaaS, SaaS), deployment models, virtualization, and cloud security. Networks is a prerequisite.

📄 Research Paper
📝 Summary: We propose federated learning for healthcare analytics avoiding centralization of patient data. Experiments show improved accuracy with differential privacy.

✅ Quick test complete. Ready for export.


In [16]:
# =====================================================================
# CELL 11: Create Download ZIP (Run Last, ~1 minute)
# =====================================================================

zip_name = "academic_summarizer_scientific.zip"
folder_to_zip = "./my_academic_summarizer_scientific"

print("\n📦 Creating downloadable archive...")

!zip -r {zip_name} {folder_to_zip} > /dev/null

print("\n" + "=" * 60)
print("🎉 FINE-TUNING COMPLETE!")
print("=" * 60)
print(f"📦 Download file: {zip_name}")
print(f"📁 Model folder zipped: {folder_to_zip}")
print("\n👉 Go to Kaggle > left sidebar > Files > click the ZIP to download.")
print("=" * 60)



📦 Creating downloadable archive...

🎉 FINE-TUNING COMPLETE!
📦 Download file: academic_summarizer_scientific.zip
📁 Model folder zipped: ./my_academic_summarizer_scientific

👉 Go to Kaggle > left sidebar > Files > click the ZIP to download.


In [17]:
# ==============================================================
# DOWNLOAD LINK GENERATOR
# ==============================================================

# move file to working so UI can access
!mv academic_summarizer_scientific.zip /kaggle/working/

from IPython.display import FileLink
print("Click the link below to download:")
FileLink("/kaggle/working/academic_summarizer_scientific.zip")


mv: 'academic_summarizer_scientific.zip' and '/kaggle/working/academic_summarizer_scientific.zip' are the same file
Click the link below to download:


/kaggle/working/academic_summarizer_scientific.zip